In [358]:
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data.dataloader
from torch.nn.functional import softmax
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import models
import os
import importlib
import utils
import matplotlib.pyplot as plt
data_dir ="./data"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [344]:
importlib.reload(models)
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\ged11\\Dropbox\\0 TUM\\Deep Learning Seminar\\HousesAndBoats\\utils.py'>

In [333]:
mnist_train = datasets.MNIST(root='./data',train=True,transform=torchvision.transforms.ToTensor(),
                            target_transform=None,download=True)
mnist_test = datasets.MNIST(root = './data', train=False,transform=torchvision.transforms.ToTensor(),
                           target_transform=None,download=True)

In [373]:
np.random.seed(143)
indices=np.arange(len(mnist_train))
indices = np.random.permutation(indices)
train_indices = indices[0:1000]
val_indices = indices[50000:]
mnist_train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=16,
                                                sampler=SubsetRandomSampler(train_indices),
                                                num_workers=4)
mnist_val_loader = torch.utils.data.DataLoader(mnist_train, batch_size=16,
                                              sampler=SubsetRandomSampler(val_indices),
                                              num_workers=4)

In [375]:
model = models.CapsNet([1,28,28],10,3)

In [370]:
images, targets = iter(mnist_train_loader).next()

In [371]:
outputs = model(images)

In [363]:
outputs.shape

torch.Size([8, 10])

In [364]:
torch.softmax(outputs,dim=1).shape

torch.Size([8, 10])

In [380]:
model.to(device)
log_every_batches = 1
log_every_epochs = 1
num_epochs = 10
train_losses = []
train_accuracies = []
val_accuracies = []
train_size = 1000
val_size = 10000
val_losses = []
loss_fn = CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=1e-3)
number_of_train_batches = len(mnist_train_loader)
for epoch in range(num_epochs):
    batch_losses = []
    num_correct = 0
    model.train()
    for i, data in enumerate(mnist_train_loader,0):
        samples, targets = data
        samples, targets = samples.to(device), targets.to(device)
        optimizer.zero_grad()
        scores = model.forward(samples)
        loss = loss_fn(scores, targets)
        loss.backward()
        optimizer.step()
        batch_losses.append(loss.item())
        y = softmax(scores, dim =1)
        y = torch.argmax(y,dim=1)
        num_correct += np.sum(y.cpu().numpy()==targets.cpu().numpy())
#         if (i+1) % log_every_batches ==0:
#             print("Epoch {0:d}, Batch {1:d}/{2:d}\nTraining Loss {3:.4f}".format(
#                 epoch+1,i+1,number_of_train_batches,loss.item()))
    epoch_loss = np.mean(batch_losses)
    train_losses.append(epoch_loss)
    epoch_accuracy = num_correct/train_size
    train_accuracies.append(epoch_accuracy)
    if (epoch+1) % log_every_epochs ==0:
        print("Epoch {0:d}; Training Loss {1:.4f}; Training Accuracy {2:.4f}".format(epoch+1,
                                                                                     epoch_loss,
                                                                                     epoch_accuracy))
#     num_correct = 0
#     model.eval()
#     for i, data in enumerate(mnist_val_loader,0):
#         batch_losses = []
        
#         samples, targets = data
#         samples, targets = samples.to(device), targets.to(device)
#         scores = model(samples)
#         loss = loss_fn(scores, targets)
#         batch_losses.append(loss.item())
#         y = softmax(scores,dim=1)
#         y = torch.argmax(y,dim=1)
#         num_correct += np.sum(y.cpu().numpy()==targets.cpu().numpy())
#     val_loss = np.mean(batch_losses)
#     val_losses.append(val_loss)
#     val_accuracy = num_correct/val_size
#     val_accuracies.append(val_accuracy)
#     if (epoch +1) % log_every_epochs ==0:
#         print("Epoch {0:d}; Validation Loss {1:.4f}; Validation Accuracy {2:.4f}".format(epoch+1,
#                                                                                          val_loss,
#                                                                                          val_accuracy))
            
            

#%%

plt.plot(np.arange(len(train_losses)),train_losses)
plt.plot(np.arange(len(val_losses)),val_losses)
plt.show()

Epoch 1; Training Loss 2.1713; Training Accuracy 0.3700


KeyboardInterrupt: 